In [2]:

# In the example below, we are going to optimize revenue produced by several classes of energy producing assets.
# If Windprice is the highest, followed by Solarprice and then Hydroprice, it implies that you want to prioritize 
# selling energy from the Wind asset due to its higher price, followed by Solar, and then finally Hydro.


In [1]:

from scipy.optimize import minimize
import numpy as np
import pandas as pd

# Set a random seed for reproducibility
np.random.seed(42)

# Generate sample data for df_opt
num_records = 100

data = {
    'wind': np.random.randint(0, 100, size=num_records),
    'solar': np.random.randint(0, 100, size=num_records),
    'hydro': np.random.randint(0, 100, size=num_records),
    'Windprice': np.random.uniform(1, 10, size=num_records),
    'Solarprice': np.random.uniform(1, 10, size=num_records),
    'Hydroprice': np.random.uniform(1, 10, size=num_records)
}

df_opt = pd.DataFrame(data)
df_opt


,wind,solar,hydro,Windprice,Solarprice,Hydroprice
0,51,25,8,8.619950,3.443886,3.147372
1,92,88,61,8.706919,9.687266,7.118603
2,14,59,36,4.640573,5.115386,7.659179
3,71,40,96,8.989931,8.578208,3.144125
4,60,28,50,8.658356,2.749420,4.399560
...,...,...,...,...,...,...
95,84,76,31,1.349513,2.149208,4.020941
96,79,2,8,3.729390,3.250148,2.527119
97,81,69,98,5.833742,6.224896,6.821635
98,52,71,18,3.939861,8.804049,4.494415


In [7]:

# Define asset types and constraints
asset_types = ['Wind', 'Solar', 'Hydro']

# Function to generate arbitrary allocations without optimization
def generate_arbitrary_allocations():
    allocations = np.random.uniform(0.05, 0.5, size=len(asset_types))
    allocations /= allocations.sum()  # Normalize to ensure the sum is 1.0
    return allocations

# Apply the arbitrary allocation function to each row in the DataFrame
arbitrary_allocations = np.array([generate_arbitrary_allocations() for _ in range(num_records)])

# Append arbitrary allocations to new columns in df_opt DataFrame
for i, asset_type in enumerate(asset_types):
    df_opt[f'Arbitrary_{asset_type}'] = arbitrary_allocations[:, i]

# Define a function to calculate revenue for a given set of allocations
def calculate_revenue(allocations, asset_prices):
    return np.sum(allocations * asset_prices)

# Define a function to optimize allocations for a given month
def optimize_allocations(month_data):
    # Extract prices for each asset type
    asset_prices = [month_data[f'{asset}price'] for asset in asset_types]

    # Sort asset types based on prices in descending order
    sorted_assets = [asset for _, asset in sorted(zip(asset_prices, asset_types), reverse=True)]

    # Define the objective function to minimize
    def objective_function(allocations):
        return -sum([allocations[i] * asset_prices[i] for i in range(len(asset_types))])

    # Define the equality constraint to ensure total allocation sums up to 1.0
    def equality_constraint(allocations):
        return sum(allocations) - 1.0

    # Create initial guesses for allocations
    initial_allocations = [1.0 / len(asset_types)] * len(asset_types)

    # Setup optimization problem with constraints
    constraints = ({'type': 'eq', 'fun': equality_constraint})
    bounds = [(0.05, 0.5)] * len(asset_types)  # Adjusted bounds for consistency

    # Solve the optimization problem
    result = minimize(objective_function, initial_allocations, method='SLSQP', bounds=bounds, constraints=constraints)

    return result.x, sorted_assets  # Return both the optimized allocations and the sorted assets


In [9]:

# Apply the optimization function to each row in the DataFrame
optimized_allocations, _ = zip(*df_opt.apply(optimize_allocations, axis=1))

# Append optimized allocations to new columns in df_opt DataFrame
for i, asset_type in enumerate(asset_types):
    df_opt[f'Optimized_{asset_type}'] = [allocations[i] for allocations in optimized_allocations]


# Calculate revenue for optimized and arbitrary allocations
df_opt['Optimized_Revenue'] = df_opt.apply(lambda row: calculate_revenue(row[['Optimized_' + asset_type for asset_type in asset_types]].values.flatten(), row[[asset_type + 'price' for asset_type in asset_types]].values), axis=1)
df_opt.head()


C:\Users\ryan_\anaconda3\Lib\site-packages\scipy\optimize\_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


,wind,solar,hydro,Windprice,Solarprice,Hydroprice,Arbitrary_Wind,Arbitrary_Solar,Arbitrary_Hydro,Optimized_Wind,Optimized_Solar,Optimized_Hydro,Optimized_Revenue
0,51,25,8,8.619950,3.443886,3.147372,0.535800,0.298329,0.165871,0.50,0.45,0.05,6.017093
1,92,88,61,8.706919,9.687266,7.118603,0.486116,0.127655,0.386229,0.45,0.50,0.05,9.117677
2,14,59,36,4.640573,5.115386,7.659179,0.432226,0.407328,0.160446,0.05,0.45,0.50,6.363542
3,71,40,96,8.989931,8.578208,3.144125,0.800264,0.092653,0.107082,0.50,0.45,0.05,8.512365
4,60,28,50,8.658356,2.749420,4.399560,0.423001,0.282080,0.294919,0.50,0.05,0.45,6.446451


In [10]:

# Function to calculate revenue for arbitrary allocations
def calculate_arbitrary_revenue(row):
    arbitrary_allocations = row[[f'Arbitrary_{asset_type}' for asset_type in asset_types]].values.flatten()
    asset_prices = row[[f'{asset_type}price' for asset_type in asset_types]].values.flatten()
    return calculate_revenue(arbitrary_allocations, asset_prices)


# Apply the arbitrary revenue calculation function to each row in the DataFrame
df_opt['Arbitrary_Revenue'] = df_opt.apply(calculate_arbitrary_revenue, axis=1)
df_opt.head()


,wind,solar,hydro,Windprice,Solarprice,Hydroprice,Arbitrary_Wind,Arbitrary_Solar,Arbitrary_Hydro,Optimized_Wind,Optimized_Solar,Optimized_Hydro,Optimized_Revenue,Arbitrary_Revenue
0,51,25,8,8.619950,3.443886,3.147372,0.535800,0.298329,0.165871,0.50,0.45,0.05,6.017093,6.168037
1,92,88,61,8.706919,9.687266,7.118603,0.486116,0.127655,0.386229,0.45,0.50,0.05,9.117677,8.218611
2,14,59,36,4.640573,5.115386,7.659179,0.432226,0.407328,0.160446,0.05,0.45,0.50,6.363542,5.318301
3,71,40,96,8.989931,8.578208,3.144125,0.800264,0.092653,0.107082,0.50,0.45,0.05,8.512365,8.325801
4,60,28,50,8.658356,2.749420,4.399560,0.423001,0.282080,0.294919,0.50,0.05,0.45,6.446451,5.735564


In [11]:

# Calculate the percentage saved by optimizing the solution
df_opt['Percentage_Saved'] = ((df_opt['Optimized_Revenue'] - df_opt['Arbitrary_Revenue']) / df_opt['Arbitrary_Revenue']) * 100


# Print the first few records of df_opt for comparison
print(df_opt.head())


   wind  solar  hydro  Windprice  Solarprice  Hydroprice  Arbitrary_Wind  \
0    51     25      8   8.619950    3.443886    3.147372        0.535800   
1    92     88     61   8.706919    9.687266    7.118603        0.486116   
2    14     59     36   4.640573    5.115386    7.659179        0.432226   
3    71     40     96   8.989931    8.578208    3.144125        0.800264   
4    60     28     50   8.658356    2.749420    4.399560        0.423001   

   Arbitrary_Solar  Arbitrary_Hydro  Optimized_Wind  Optimized_Solar  \
0         0.298329         0.165871            0.50             0.45   
1         0.127655         0.386229            0.45             0.50   
2         0.407328         0.160446            0.05             0.45   
3         0.092653         0.107082            0.50             0.45   
4         0.282080         0.294919            0.50             0.05   

   Optimized_Hydro  Optimized_Revenue  Arbitrary_Revenue  Percentage_Saved  
0             0.05           6.01

In [3]:

# This prioritization concept aligns with maximizing revenue by selling energy from the asset with the highest price 
# first, and then the second highest next, and finally the third last. We can easily see how the revenue is optimized
# (increased) over the methodology of randomly selling energy from assets arbitrarily.
